In [93]:
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster
import pandas as pd
import geopandas as gpd
import importlib
import models
from sqlalchemy import text
importlib.reload(models)
from models import Route, get_conn
pd.set_option('display.expand_frame_repr', False) # display full data in terminal

engine, Session = get_conn()

In [94]:
# Fetch Tick, Route and Trail data from DB, organize into geo_data (list of lat/long, name)
with Session() as session:
    statement = session.query(Route).statement
    routes_df = pd.read_sql(statement, con=session.bind)

    raw_ticks_sql = 'SELECT ticked_routes.route, routes.area_latitude, routes.area_longitude FROM ticked_routes INNER JOIN routes on ticked_routes.routeid = routes.routeid'
    sql_result = session.execute(text(raw_ticks_sql))

    rows = sql_result.fetchall()
    columns = sql_result.keys()
    ticks_df = pd.DataFrame(rows, columns=columns)

    # ticks_df = pd.read_sql(tick_statement, con=session.bind)

    trail_geo_df = gpd.read_postgis("SELECT * FROM trails", con=engine, geom_col='geometry')

route_geo_data = []
ticks_geo_data = []

# ESCAPE BACKTICKS - they mess up the HTML generated by folium.
for idx, route in routes_df.iterrows():
    route_geo_data.append({'latitude': route.area_latitude, 'longitude': route.area_longitude, 'name': route.route_name.replace("`", "&#96;")})

for idx, tick in ticks_df.iterrows():
    ticks_geo_data.append({'latitude': tick.area_latitude, 'longitude': tick.area_longitude, 'name': tick.route.replace("`", "&#96;")})


In [ ]:
# Create and Render the Map of Climbing Areas (clustered) and all Trail data in the DB.
center = [trail_geo_df['latitude'].mean(), trail_geo_df['longitude'].mean()]
map = folium.Map(location=center, zoom_start=5)

# Add GaiaGPS Trails
for _, trail in trail_geo_df.iterrows():
    folium.GeoJson(trail['geometry'], name=trail['title'], tooltip=trail['title'], color='red').add_to(map)

## Ad MP route locations

mp_cluster = MarkerCluster().add_to(map)

for point in ticks_geo_data:
    folium.Marker(
        location=[point['latitude'], point['longitude']],
        tooltip=point['name'],
        icon=folium.Icon(color='blue')
    ).add_to(mp_cluster)

# map.save('html_maps/gaia_trails_and_climbing_areas.html')
map